In [ ]:
#bibliotecas

In [ ]:
estados = {"s0", "s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8", "s9", "s10", "s11", "s12", "s13", "s14", "s15"}
acoes = {"cima", "baixo", "esquerda", "direita", "atirar", "agarrar"}
tabela_recompensas = {}
estado_inicial = "s0"